<a href="https://colab.research.google.com/github/ThierrryScotto/Optuna-XGBoost-hyperparameter/blob/main/Optuna_XGBoost_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import sklearn

sns.set()
%config InlineBeckend.figure_formar = 'retina'
warnings.filterwarnings("ignore")

## Loading the boston dataset

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [4]:
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['target']  = boston.target

In [5]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [6]:
X = df.iloc[:, df.columns != 'target']
y = df.target

### Scaling data

In [7]:
from sklearn.preprocessing import StandardScaler
se = StandardScaler()
X = se.fit_transform(X)

### Split data into training and test

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

**Converting the training and test data into xgboost optimized matrix for efficiency**

In [10]:
df_train = xgb.DMatrix(X_train, y_train)
df_test = xgb.DMatrix(X_test, y_test)

### Testing a base model

In [21]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, ShuffleSplit

cv_ = KFold(n_splits=10)
xg_reg = xgb.XGBRFRegressor()
scores = cross_val_score(xg_reg, X_train, y_train, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=cv_)

In [22]:
print(np.mean(scores), np.std(scores))
print(scores)

-4.690851864521107 0.9137364320425729
[-4.46876379 -4.44228761 -4.73628169 -5.57790661 -3.89063134 -5.08357565
 -5.49483956 -3.32361937 -6.36116408 -3.52944895]


#### Creating a scoring funtion with the help of XGBOOST train inbuilt early stopping
here we are not tuning the n_estimators with the help of optuna

we will tune the n_estimators with the help of early_stopping by using the xgboost.train

In [35]:
import re
def return_rmse(params):
  model = xgb.train(params, df_train, num_boost_round=600, evals=[(df_test, 'eval')], early_stopping_rounds=20, verbose_eval=0)
  result = model.eval(df_test)
  result = np.float(re.search(r'[\d.]+$', result).group(0))
  print(result)
  return result

#### importing optuna

In [25]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.8 MB/s eta 0:00:00


In [28]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [29]:
def objective(trial):
    param = {
                # "n_estimators" : trial.suggest_int('n_estimators', 0, 500),
                'max_depth':trial.suggest_int('max_depth', 3, 5),
                'reg_alpha':trial.suggest_uniform('reg_alpha',0,6),
                'reg_lambda':trial.suggest_uniform('reg_lambda',0,2),
                'min_child_weight':trial.suggest_int('min_child_weight',0,5),
                'gamma':trial.suggest_uniform('gamma', 0, 4),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.05,0.5),
                'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9),
                'subsample':trial.suggest_uniform('subsample',0.4,0.9),

                'nthread' : -1
            }
    return(return_rmse(param)) # this will return the rmse score 

**calling the optuna study**

In [36]:
stady1 = optuna.create_study(direction='minimize', sampler=TPESampler())
stady1.optimize(objective, n_trials=1050, show_progress_bar=True)

[I 2023-01-19 00:51:19,145] A new study created in memory with name: no-name-17084d8c-261a-4779-b94d-9c37a10abdf8


  0%|          | 0/1050 [00:00<?, ?it/s]

3.20587
[I 2023-01-19 00:51:19,399] Trial 0 finished with value: 3.20587 and parameters: {'max_depth': 4, 'reg_alpha': 2.504865479717795, 'reg_lambda': 1.9844890350875033, 'min_child_weight': 0, 'gamma': 0.5943381121429985, 'learning_rate': 0.12901055064575767, 'colsample_bytree': 0.49391109018192864, 'subsample': 0.7810724330643195}. Best is trial 0 with value: 3.20587.
3.376091
[I 2023-01-19 00:51:19,541] Trial 1 finished with value: 3.376091 and parameters: {'max_depth': 3, 'reg_alpha': 1.8792914209412455, 'reg_lambda': 1.3397859041484956, 'min_child_weight': 4, 'gamma': 2.077989481314049, 'learning_rate': 0.2548900472548867, 'colsample_bytree': 0.7043026565592678, 'subsample': 0.8532257076573416}. Best is trial 0 with value: 3.20587.
3.649873
[I 2023-01-19 00:51:19,651] Trial 2 finished with value: 3.649873 and parameters: {'max_depth': 4, 'reg_alpha': 1.0593610673844511, 'reg_lambda': 1.2965113592723292, 'min_child_weight': 4, 'gamma': 2.1976269348303106, 'learning_rate': 0.333967

In [37]:
return_rmse(stady1.best_params)

2.754399


2.754399